# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), Dec 2, 2017**

A brief introduction to basic usage of `DataFrames`. Tested under version `0.11`.
I will try to keep it up to date as the package evolves.

In [1]:
using DataFrames # load package

## Working with CategoricalArrays

### Constructor

In [2]:
x = categorical(["A", "B", "B", "C"]) # unordered

4-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [3]:
y = categorical(["A", "B", "B", "C"], ordered=true) # ordered, by default order is sorting order

4-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [4]:
z = categorical(["A","B","B","C", missing]) # unordered with missings

5-element CategoricalArrays.CategoricalArray{Union{Missings.Missing, String},1,UInt32}:
 "A"    
 "B"    
 "B"    
 "C"    
 missing

In [5]:
c = cut(1:10, 5) # ordered, into equal counts, possible to rename labels and give custom breaks

10-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "[1.0, 2.8)" 
 "[1.0, 2.8)" 
 "[2.8, 4.6)" 
 "[2.8, 4.6)" 
 "[4.6, 6.4)" 
 "[4.6, 6.4)" 
 "[6.4, 8.2)" 
 "[6.4, 8.2)" 
 "[8.2, 10.0]"
 "[8.2, 10.0]"

In [6]:
by(DataFrame(x=cut(randn(100000), 10)), :x, d -> DataFrame(n=nrow(d)), sort=true) # just to make sure it works right

,x,n
1,"[-4.12824, -1.28751)",10000
2,"[-1.28751, -0.840468)",10000
3,"[-0.840468, -0.525527)",10000
4,"[-0.525527, -0.258704)",10000
5,"[-0.258704, -0.00449616)",10000
6,"[-0.00449616, 0.249299)",10000
7,"[0.249299, 0.525108)",10000
8,"[0.525108, 0.845545)",10000
9,"[0.845545, 1.27822)",10000
10,"[1.27822, 4.335]",10000


In [7]:
v = categorical([1,2,2,3,3]) # contains integers not strings

5-element CategoricalArrays.CategoricalArray{Int64,1,UInt32}:
 1
 2
 2
 3
 3

In [8]:
Vector{Union{String, Missing}}(z) # sometimes you need to convert back to a standard vector

5-element Array{Union{Missings.Missing, String},1}:
 "A"    
 "B"    
 "B"    
 "C"    
 missing

### Managing levels

In [9]:
isordered.([x,y,z,c,v]) # chcek if array is orderd

5-element BitArray{1}:
 false
  true
 false
  true
 false

In [10]:
ordered!(x, true), isordered(x) # make x ordered

(CategoricalArrays.CategoricalString{UInt32}["A", "B", "B", "C"], true)

In [11]:
ordered!(x, false), isordered(x) # and unordered again

(CategoricalArrays.CategoricalString{UInt32}["A", "B", "B", "C"], false)

In [12]:
levels.([x,y,z,c,v]) # list levels

5-element Array{Array{T,1} where T,1}:
 String["A", "B", "C"]                                                        
 String["A", "B", "C"]                                                        
 String["A", "B", "C"]                                                        
 String["[1.0, 2.8)", "[2.8, 4.6)", "[4.6, 6.4)", "[6.4, 8.2)", "[8.2, 10.0]"]
 [1, 2, 3]                                                                    

In [13]:
unique.([x,y,z,c,v]) # missing will be included

5-element Array{Array{T,1} where T,1}:
 String["A", "B", "C"]                                                        
 String["A", "B", "C"]                                                        
 Union{Missings.Missing, String}["A", "B", "C", missing]                      
 String["[1.0, 2.8)", "[2.8, 4.6)", "[4.6, 6.4)", "[6.4, 8.2)", "[8.2, 10.0]"]
 [1, 2, 3]                                                                    

In [14]:
y[1] < y[2] # can compare as y is ordered

true

In [15]:
v[1] < v[2] # not comparable, v is unordered although it contains integers

LoadError: [91mArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this[39m

In [16]:
levels!(y, ["C", "B", "A"]) # you can reorder levels, mostly useful for ordered CategoricalArrays

4-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [17]:
y[1] < y[2] # observe that the order is changed

false

In [18]:
levels!(z, ["A", "B"]) # you have to specify all levels that are present

LoadError: [91mArgumentError: cannot remove level "C" as it is used at position 4 and allow_missing=false.[39m

In [19]:
levels!(z, ["A", "B"], allow_missing=true) # unless the underlying array allows for missings and force removal of levels

5-element CategoricalArrays.CategoricalArray{Union{Missings.Missing, String},1,UInt32}:
 "A"    
 "B"    
 "B"    
 missing
 missing

In [20]:
z[1] = "B"
z # now z has only "B" entries

5-element CategoricalArrays.CategoricalArray{Union{Missings.Missing, String},1,UInt32}:
 "B"    
 "B"    
 "B"    
 missing
 missing

In [21]:
levels(z) # but it remembers the levels it had (the reason is mostly performance)

2-element Array{String,1}:
 "A"
 "B"

In [22]:
droplevels!(z) # this way we can clean it up
levels(z)

1-element Array{String,1}:
 "B"

### Data manipulation

In [23]:
y, levels(y)

(CategoricalArrays.CategoricalString{UInt32}["A", "B", "B", "C"], String["C", "B", "A"])

In [24]:
y[2] = "0"
y, levels(y) # new level added at the end (this will be the largest level in ordered case)

(CategoricalArrays.CategoricalString{UInt32}["A", "0", "B", "C"], String["C", "B", "A", "0"])

In [25]:
v, levels(v)

(CategoricalArrays.CategoricalValue{Int64,UInt32}[1, 2, 2, 3, 3], [1, 2, 3])

In [26]:
v[1] + v[2] # even though underlying data is Int, we cannot operate on it

LoadError: [91mMethodError: no method matching +(::CategoricalArrays.CategoricalValue{Int64,UInt32}, ::CategoricalArrays.CategoricalValue{Int64,UInt32})[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

In [27]:
Vector{Int}(v) # you have either to retrieve the data by conversion (may be expensive)

5-element Array{Int64,1}:
 1
 2
 2
 3
 3

In [28]:
get(v[1]) + get(v[2]) # or get a single value

3

In [29]:
get.(v) # this will work for arrays witout missings

5-element Array{Int64,1}:
 1
 2
 2
 3
 3

In [30]:
get.(z) # but will fail on missing values

LoadError: [91mMethodError: no method matching get(::Missings.Missing)[0m
Closest candidates are:
  get([91m::ObjectIdDict[39m, [91m::ANY[39m, [91m::ANY[39m) at associative.jl:434
  get([91m::Base.EnvHash[39m, [91m::AbstractString[39m, [91m::Any[39m) at env.jl:79
  get([91m::ZMQ.Context[39m, [91m::Integer[39m) at D:\Software\JULIA_PKG\v0.6\ZMQ\src\ZMQ.jl:133
  ...[39m

In [31]:
Vector{Union{String, Missing}}(z) # you have to do the conversion

5-element Array{Union{Missings.Missing, String},1}:
 "B"    
 "B"    
 "B"    
 missing
 missing

In [32]:
z[1]*z[2], z.^2 # the only exception are CategoricalArrays based on String - you can operate on them normally

("BB", Any["BB", "BB", "BB", missing, missing])

In [33]:
recode([1,2,3,4,5,missing], 1=>10) # recode some values in an array; has also in place recode! equivalent

6-element Array{Union{Int64, Missings.Missing},1}:
 10       
  2       
  3       
  4       
  5       
   missing

In [34]:
recode([1,2,3,4,5,missing], "a", 1=>10, 2=>20) # here we provided a default value for not mapped recodings

6-element Array{Any,1}:
 10       
 20       
   "a"    
   "a"    
   "a"    
   missing

In [35]:
recode([1,2,3,4,5,missing], 1=>10, missing=>"missing") # to recode Missing you have to do it explicitly

6-element Array{Any,1}:
 10         
  2         
  3         
  4         
  5         
   "missing"

In [36]:
t = categorical([1:5; missing])
t, levels(t)

(Union{CategoricalArrays.CategoricalValue{Int64,UInt32}, Missings.Missing}[1, 2, 3, 4, 5, missing], [1, 2, 3, 4, 5])

In [37]:
recode!(t, [1,3]=>2)
t, levels(t) # note that the levels are dropped after recode

(Union{CategoricalArrays.CategoricalValue{Int64,UInt32}, Missings.Missing}[2, 2, 2, 4, 5, missing], [2, 4, 5])

In [39]:
t = categorical([1,2,3], ordered=true)
levels(recode(t, 2=>0, 1=>-1)) # and if you introduce a new levels they are added at the end in the order of appearance

3-element Array{Int64,1}:
  3
  0
 -1

In [42]:
t = categorical([1,2,3,4,5], ordered=true) # when using default a new ordering of levels is introduced
levels(recode(t, 300, [1,2]=>100, 3=>200))

3-element Array{Int64,1}:
 100
 200
 300